<a href="https://www.kaggle.com/code/rebeccapringle/credit-card-fraud-detection-model?scriptVersionId=117761008" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

The dataset contains data to train a supervised machine learning model and test data. I have improved the model by joining datasets but no feature engineering or EDA has been done to further improve the model. 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
df_train = pd.read_csv('/kaggle/input/playground-series-s3e4/train.csv', index_col = 0)
df_test = pd.read_csv('/kaggle/input/playground-series-s3e4/test.csv', index_col = 0)
orig = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv', index_col = 0) 

In [3]:
# no of nonfraudulent events
df_train[df_train['Class'] == 0].shape[0]

218660

In [4]:
# number of detected fraud events
df_train[df_train['Class'] == 1].shape[0]

469

This is very small in comparison which suggests the model to contains lots of bias. I am going to add data that detected fraud from the original dataset to help the model train for better accuracy. 

In [5]:
df_orig_class_1 = orig[orig['Class']==1]
df_train = pd.concat([df_train, df_orig_class_1])

Prepare the data for applying to the model.

In [6]:
df_train.drop('Time', axis=1, inplace=True)
df_test.drop('Time', axis=1, inplace=True)
df_train.index = np.arange(df_train.shape[0])

In [7]:
X = df_train.drop('Class', axis=1)
y = df_train['Class'].astype('int')

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.1, random_state=10, shuffle=True)

Choose the model and hypertune the parameters for the best outcome. This is commented out for easier running of the code. 

In [9]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(n_estimators = 55, learning_rate = 0.45)
ada_clf.fit(X_train, y_train)

#for n in [40,45,50,55,60]:
#    for l in [0.4,0.425,0.45,0.475, 0.5]:
#        ada_clf = AdaBoostClassifier(n_estimators = n, learning_rate = l)
 #       ada_clf.fit(X_train, y_train)
 #       ada_pred_proba = ada_clf.predict_proba(X_val)[:, 1]
 #       a = roc_auc_score(y_val, ada_pred_proba)
  #      print("N: %.2f Rate: %.2f Score: %.4f" %(n,l,a))
        

AdaBoostClassifier(learning_rate=0.45, n_estimators=55)

Find the metric rating that is used for this competition and see the score. One metric doesn't give entire clarity over the success of the model. In the case of fraud overestimating would be more successfull than letting lots of fraud cases go without being detected, despite having slightly lower metrics. 

In [10]:
from sklearn.metrics import roc_auc_score
ada_pred_proba = ada_clf.predict_proba(X_val)[:, 1]

print('Adaboost\'s roc_auc_score: ', roc_auc_score(y_val, ada_pred_proba))

Adaboost's roc_auc_score:  0.9058515379625437


In [11]:
# confusion matrix
from sklearn.metrics import confusion_matrix

y_pred_ada = ada_clf.predict(X_val)
confusion_matrix(y_val, y_pred_ada)

array([[21858,     7],
       [   62,    36]])

Apply to test data and output to submission file. 

In [12]:
ada_preds = ada_clf.predict_proba(df_test)[:, 1]
submission = pd.DataFrame({
    'id': df_test.index,
    'Class': ada_preds
})
submission.head()

,id,Class
0,219129,0.437663
1,219130,0.427716
2,219131,0.423128
3,219132,0.443832
4,219133,0.423881


In [13]:
submission.to_csv('./submission.csv', index = False)